In [1]:
%load_ext autoreload
%autoreload 2
from utils.predict_np import predict_from_np
from scipy.stats import norm
from shapley.shapley_fda import ShapleyFda
from shapley.shaple_fda_mean import ShapleyFdaMean
from skfda.misc import inner_product
from skfda.ml.regression import KNeighborsRegressor
from skfda.representation.basis import BSplineBasis
from skfda.representation.grid import FDataGrid
import numpy as np

In [ ]:
n_basis_representation = 100
ini = 0
end = 1
std = 0.02
total_fun = 100
mu_ini_1 = 0.2
mu_end_1 = 0.3
h_1 = (mu_end_1 - mu_ini_1)/total_fun
mu_ini_2 = 0.7
mu_end_2 = 0.8
h_2 = (mu_end_2 - mu_ini_2)/total_fun
domain_range = (ini, end)
basis_bsplines = BSplineBasis(n_basis=n_basis_representation, domain_range=domain_range)
step = 0.01
abscissa_points = np.arange(ini, end + step, step)
total_abscissa_points = abscissa_points.shape[0]
mean_1 = np.arange(mu_ini_1, mu_end_1, h_1)
mean_2 = np.arange(mu_ini_2, mu_end_2, h_2)
mean_all = np.concatenate((mean_1, mean_2))
num_functions = mean_all.shape[0]
X_simulated = np.empty(shape=(num_functions, total_abscissa_points))
i = 0
for mean in mean_all:
    normal_rv = norm(loc=mean, scale=std)
    normal_pdf = normal_rv.pdf(abscissa_points)
    if mean >= 0.5:
        normal_pdf = -normal_pdf
    X_simulated[i, :] = normal_pdf
    i += 1
X_simulated_grid = FDataGrid(
    data_matrix=X_simulated,
    grid_points=abscissa_points,
    domain_range=domain_range
)
X_basis_bsplines = X_simulated_grid.to_basis(basis_bsplines)

In [ ]:
zzz = X_simulated_grid.plot()

In [ ]:
zzz = X_basis_bsplines.plot()

In [ ]:
fun = np.ones(shape=abscissa_points.shape)
fun_grid = FDataGrid(
    data_matrix= fun,
    grid_points=abscissa_points, 
    domain_range=(ini, end)
)
zzz = fun_grid.plot()

In [ ]:
np.random.seed(12345)
y = inner_product(X_simulated_grid, fun_grid) + np.random.normal(scale=0.001, size=num_functions)
y

In [ ]:
# KNN
neigh = KNeighborsRegressor()
neigh.fit(X_basis_bsplines, y)

In [ ]:
neigh.predict(X_basis_bsplines)

In [ ]:
# Transform predict function to use a numpy array as input
pred_neigh = predict_from_np(
    grid_points=abscissa_points,
    domain_range=domain_range,
    basis=X_basis_bsplines.basis,
    predict_fn=neigh.predict
)

In [ ]:
num_intervals = 20
num_permutations = 100
shapley_fda_neigh = ShapleyFda(
    predict_fn=pred_neigh,
    X=X_simulated,
    abscissa_points=abscissa_points,
    target=y,
    domain_range=domain_range,
    verbose=False,
)
values_shapley_lm = shapley_fda_neigh.compute_shapley_value(
    num_intervals=num_intervals,
    num_permutations=num_permutations
)

In [ ]:
shapley_fda_neigh.plot()